In [1]:
%pip install -e ../cybercomp-python-sdk

Obtaining file:///Users/yasith/projects/cybershuttle/cybercompute/cybercomp-python-sdk
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for cybercomp (pyproject.toml) ... done
  Created wheel for cybercomp: filename=cybercomp-0.1.0-0.editable-py3-none-any.whl size=1520 sha256=b06fb212d963c9aff0757be8cd59e80b71bd015e504b0eb2a88bb1d338265d09
  Stored in directory: /private/var/folders/_n/fcf6nx4j67gbbt4_8mjqxdc80000gn/T/pip-ephem-wheel-cache-rd89h765/wheels/c2/8c/84/1bb5e7b975cbcb042fc23f5041235abbcbe1c8dc0b2665d06a
Successfully built cybercomp
  Attempting uninstall: cybercomp
    Found existing installation: cybercomp 0.1.0
    Uninstalling cybercomp-0.1.0:
      Successfully uninstalled cybercomp-0.1.0
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from pathlib import Path

from cybercomp import Completions

# generate statically typed objects to define experiments
completions = Completions(
    server_url="http://127.0.0.1:8765",
    base_path=Path("./completions"),
)
completions.sync()

In [4]:
from cybercomp import (
    Experiment,
    Runtime,
    Parameter as P,
    Hyperparameter as H,
    Observation as O,
)
# now you can import autocompletion classes for use with experiments
from completions import models, engines
from completions.types import *

In [ ]:
# a simple experiment using the generated types

# TODO remove Unit part, infer Unit/Sweep/Sequence based on input args

m1, e1 = models.make_network, engines.bazhenov_labs_engine
ex1 = Experiment.Unit(
    name="make_bazhenov_network",
    model=models.make_network,
    engine=engines.bazhenov_labs_engine,
    parameters="samples/sleep_stage_transition/network.cfg",
    observations=[neuro_connection_info, neuro_field_file_0],
    hyperparameters={
        neuro_current_params: "samples/sleep_stage_transition/params.txt",
    },
)

runtime = Runtime()
ex1.run()
ex1.fetch(runtime, {})
print(isinstance(ex1, Experiment))

In [ ]:
# a simple experiment that requires observations generated from ex1
m2, e2 = models.stim_current, engines.bazhenov_labs_engine
ex2 = Experiment.Unit(
    name="simulate_bazhenov_network",
    model=m2,
    engine=e2,
    parameters={
        m2.neuro_current_params: "samples/sleep_stage_transition/params.txt",
        m2.neuro_connection_info: "samples/sleep_stage_transition/connection_info",
    },
    hyperparameters={
        e2.neuro_current_params: "samples/sleep_stage_transition/params.txt",
    },
    observations={
        m2.neuro_time_in: "samples/sleep_stage_transition/neuro_time_in",
    },
)
print(isinstance(ex2, Experiment))


Experiment(
    models.make_network > models.stim_current,
    engines.bazhenov_labs_engine
)

In [ ]:
# define a runtime for the experiments
runtime = Runtime()

# experiments can be chained into a larger experiment
# here, the parameters and observations of the second experiment are resolved
# from the second experiment, and can be overridden.
# the operator validates the parameters and warns if any parameter is missing.
ex1 > ex2 > (ex3 | ex4 | ex5) > ex6


ex6 = () # ex1 > ex2 > (ex3 | ex4 | ex5) > ex6 must be inferred

seq1 = Experiment.Sequence("bazhenov_network", ex1, ex2)
print(isinstance(seq1, Experiment))

In [7]:
range_P = [
    P.Range(models.stim_current.neuro_connection_info, 0.1, 1.0, 10),
    P.Range(models.stim_current.neuro_current_params, 0.1, 1.0, 10),
]
range_H = [
    H.Range(engines.bazhenov_labs_engine.neuro_current_params, 0.1, 1.0, 10),
]

In [ ]:
sweep1 = Experiment.Sweep("bazhenov_network_grid_search", (seq1,), range_P, range_H, []) # change syntax
sweep1.run(runtime)
exChain_obs = sweep1.fetch(runtime)

In [ ]:
# run sweep
for p in range_P:
    for h in range_H:
        

In [ ]:
ex12 = Collection(
    name="simulate_bhazenov_network",
    models=[models.make_network, models.stim_current],
    engine=engines.bazhenov_labs_engine,
    parameters=[
        models.make_network.neuro_network_config("samples/sleep_stage_transition/params.txt"),
        models.stim_current.neuro_current_params("samples/sleep_stage_transition/connection_info")
    ],
    parameters={
        "neuro_network_config": P("samples/sleep_stage_transition/network.cfg"),
        "neuro_current_params": P("samples/sleep_stage_transition/params.txt"),
    },
    hyperparameters={
        "neuro_current_params": H("samples/sleep_stage_transition/params.txt"),
    },
    observations={
        "neuro_time_cx6": O("observations/time_cx6")
    },
)

runtime = Runtime()
ex2.execute(runtime)
ex2_obs = ex2.gather_observations(runtime)

## POC - double pendulum simulation
## Real - replicate simone's experiments


# the intersecting observations are piped into the params of the next experiment
# only the intersecting observations are shown in autocompletion

experiment_2 = create_new_experiment_from(experiment_1)
experiment_2.name = "<>"
experiment_2.parameters["neuro/network_config"] = "new-value"
experiment.check_existing()

experiment.observations

experiemnt.validate()
experiment.run(
hpc_recipe={}
)

# !!! [collection] - a set of experiments with common observations

# observations may be a huge list, so need not provide everytime when its

implictly discoverable

# to get experiments run with different observations

collection = create_collection(
model="name_of_model",
parameters={
"neuro/network_config": [],
},
)

# the collection experiments are pulled from the db

collection = create_collection(
model=["model1", "model2", ...],
parameters={
"neuro/network_config": [],
},
observations={
"neuro/network_config": [],
},
) -> [list of experiments]

# all experiments sharing the same observations

collection = create_collection(
observations={
"neuro/network_config": [],
},
) -> [list of experiments]

collection = [experiment_1, experiment_2]

# experiment collection

# example of experiment chaining (top-to-bottom mro)

# example 1

experiment = Experiment(
experiment_1,
experiment_2,
)

# example 2

experiment = Experiment(
[experiment_2, experiment_3, .....], #
experiment_1, #
)

#

[
exp2 -> exp1,
exp3 -> exp1,
]

# example 3

experiment = Experiment(
[experiment_2, ...collection.experiments],
experiment_1,
)

In [ ]:
# analysis part =========================

# takes a collection as input,

# and runs some function over the observations on that

# collection

# a primitive form of experiment using a collection of experiments as input

analysis = Analysis(
collection=[],
function={

    }

)

analysis = experiment